# TopicBank: Bank Creation Experiment

Here we are going to collect interpretable topics (automatically, using topic coherence) from multiple model training.
These topics constitute *topic bank*.
And then the topic bank is going to be used for estimating topic models quality in the notebook [TopicBank-Experiment: Model Validation](TopicBank-Experiment-ModelValidation.ipynb).

The process is repeated for several datasets (some of them are already downloadable using [TopicNet](https://github.com/machine-intelligence-laboratory/TopicNet) library).

# Contents<a id="contents"></a>

* [Data](#data)
    * [Coocs](#coocs)
        * [Lower Memory Consumption (or a Bit of Shamanism. Part 1)](#optimizing-memory)
    * [Documents for Coherence Scores](#docs-for-cohs)
        * [Lower Time Consumption in Case of Big Datasets (or a Bit of Shamanism. Part 2)](#optimizing-time)
* [Experiment](#experiment)
    * [Scores](#scores)
    * [Bank Creation](#bank-creation)
* [Postprocessing](#postprocessing)

In [ ]:
# General imports

import dill
import itertools
import json
import numpy as np
import os
import pandas as pd
import sys

from enum import Enum
from scipy.stats import gaussian_kde
from matplotlib import pyplot as plt
from tqdm import tqdm
from typing import (
    Dict,
    Iterable,
)

%matplotlib inline

In [ ]:
# Making `topnum` module visible for Python

sys.path.insert(0, '..')

In [ ]:
# Optimal number of topics

from topicnet.cooking_machine import Dataset

from topnum.data.vowpal_wabbit_text_collection import VowpalWabbitTextCollection
from topnum.scores import (
    IntratextCoherenceScore,
    SparsityPhiScore,
    SparsityThetaScore,
    SimpleTopTokensCoherenceScore,
    SophisticatedTopTokensCoherenceScore,
)
from topnum.scores._base_coherence_score import (
    SpecificityEstimationMethod,
    TextType,
    WordTopicRelatednessType,
)
from topnum.scores.intratext_coherence_score import ComputationMethod
from topnum.search_methods import TopicBankMethod
from topnum.search_methods.topic_bank.topic_bank import TopicBank
from topnum.search_methods.topic_bank.one_model_train_funcs import (
    default_train_func,

    # Functions below are not used (but could have been)

#     regularization_train_func,
#     specific_initial_phi_train_func,
#     background_topics_train_func,

)

## Data<a id="data"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Loading data from disk, creating batches, dictionary, gathering cooccurrence statistics...

In [ ]:
DATA_FOLDER_PATH = os.path.join('.', 'data')

In [ ]:
sorted(os.listdir(DATA_FOLDER_PATH))

['.ipynb_checkpoints',
 '20NG_natural_order.csv',
 '20NG_natural_order_250.csv',
 '20NG_natural_order_40.csv',
 '20NG_natural_order_45.csv',
 '20NG_natural_order_50.csv',
 '20NG_natural_order_55.csv',
 '20NG_natural_order_60.csv',
 '20NG_natural_order_65.csv',
 'AG_News.csv',
 'AG_News_100.csv',
 'AG_News_1000.csv',
 'AG_News_10000.csv',
 'AG_News_250.csv',
 'AG_News_500.csv',
 'Brown.csv',
 'Brown_10.csv',
 'Brown_8.csv',
 'Brown_80.csv',
 'Brown_9.csv',
 'Habrahabr.csv',
 'Habrahabr_100.csv',
 'Habrahabr_15.csv',
 'Habrahabr_50.csv',
 'PScience.csv',
 'PostNauka_natural_order.csv',
 'Post_Science',
 'Post_Science_12.csv',
 'Post_Science_15.csv',
 'Post_Science_18.csv',
 'Post_Science_20.csv',
 'Post_Science_65.csv',
 'Reuters.csv',
 'Reuters_20.csv',
 'Reuters_50.csv',
 'Reuters_60.csv',
 'Reuters_80.csv',
 'Watan2004.csv',
 'Watan2004_10.csv',
 'Watan2004_100.csv',
 'Watan2004_12.csv',
 'Watan2004_15.csv',
 'Watan2004_250.csv']

In [ ]:
class DatasetName(Enum):
    POSTNAUKA = 'Post_Science'
    REUTERS = 'Reuters'
    BROWN = 'Brown'
    TWENTY_NEWSGROUPS = '20NG_natural_order'
    AG_NEWS = 'AG_News'
    WATAN = 'Watan2004'
    HABRAHABR = 'Habrahabr'

In [ ]:
DATASET_NAME_TO_DATASET_FILE_PATH = {
    DatasetName.POSTNAUKA: os.path.join(
        DATA_FOLDER_PATH, 'PostNauka_natural_order.csv'
    ),
    DatasetName.REUTERS: os.path.join(
        DATA_FOLDER_PATH, 'Reuters.csv'
    ),
    DatasetName.BROWN: os.path.join(
        DATA_FOLDER_PATH, 'Brown.csv'
    ),
    DatasetName.TWENTY_NEWSGROUPS: os.path.join(
        DATA_FOLDER_PATH, '20NG_natural_order.csv'
    ),
    DatasetName.AG_NEWS: os.path.join(
        DATA_FOLDER_PATH, 'AG_News.csv'
    ),
    DatasetName.WATAN: os.path.join(
        DATA_FOLDER_PATH, 'Watan2004.csv'
    ),
    DatasetName.HABRAHABR: os.path.join(
        DATA_FOLDER_PATH, 'Habrahabr.csv'
    ),
}

In [ ]:
DATASET_NAME = DatasetName.POSTNAUKA  # select a dataset here

DATASET_FILE_PATH = DATASET_NAME_TO_DATASET_FILE_PATH[DATASET_NAME]

Checking if all OK with data, what modalities does the collection have.

In [ ]:
! head -n 2 $DATASET_FILE_PATH

id,raw_text,vw_text
29998.txt,материал отрицательный показатель преломление физик виктор веселаго распространение свет вещество фазовый групповой скорость метаматериалы различаться фазовый групповой скорость каков физика распространение свет вещество находить применение материал отрицательный показатель преломление рассказывать доктор физикоматематический наука виктор веселаго скорость распространяться энергия вещество обычно говорить излучение распространяться вещество со скорость n раз маленький n коэффициент преломление вещество коэффициент преломление n отношение скорость свет скорость распространение излучение вещество обычно уточняться распространяться распространение энергия распространение импульс происходить различный закон энергия распространяться со скорость называться групповой скорость много скорость свет эйнштейн сформулировать самый больший скорость излучение скорость свет кмс импульс распространяться фазовый скорость сколь угодно много скорость свет скорость входить соо

In [ ]:
def get_dataset_internals_folder_path(dataset_name: DatasetName) -> str:
    return os.path.join('.', dataset_name.value + '__internals')

In [ ]:
DATASET_INTERNALS_FOLDER_PATH = get_dataset_internals_folder_path(DATASET_NAME)

In [ ]:
DATASET_INTERNALS_FOLDER_PATH

'./Post_Science__internals'

In [ ]:
%%time

# If using really big datasets (like Habrahabr),
# one may need to set this equal `False`
KEEP_DATASET_IN_MEMORY = True

DATASET = Dataset(
    DATASET_FILE_PATH,
    internals_folder_path=DATASET_INTERNALS_FOLDER_PATH,
    keep_in_memory=KEEP_DATASET_IN_MEMORY,
)

Looking what is inside dataset's folder

In [ ]:
os.listdir(DATASET_INTERNALS_FOLDER_PATH)

['vw.txt',
 'vocab.txt',
 'dict.dict.txt',
 'new_ppmi_tf_',
 'ppmi_tf_',
 'dict.dict',
 'cooc_values.json',
 'batches']

Creating batches

In [ ]:
DATASET.get_batch_vectorizer()

artm.BatchVectorizer(data_path="./Post_Science__internals__test/batches", num_batches=4)

In [ ]:
os.listdir(DATASET_INTERNALS_FOLDER_PATH)

['vw.txt',
 'vocab.txt',
 'dict.dict.txt',
 'new_ppmi_tf_',
 'ppmi_tf_',
 'dict.dict',
 'cooc_values.json',
 'batches']

In [ ]:
if KEEP_DATASET_IN_MEMORY:
    DOCUMENTS = list(DATASET._data.index)
else:
    DOCUMENTS = list(DATASET._data_index)

NUM_DOCUMENTS = len(DOCUMENTS)

print(f'Num documents: {NUM_DOCUMENTS}')

Num documents: 3446


Let's look at some text samples

In [ ]:
DATASET._data.head()

,id,raw_text,vw_text
id,,,
29998.txt,29998.txt,материал отрицательный показатель преломление ...,29998.txt |@word материал отрицательный показа...
7770.txt,7770.txt,культурный код экономика экономист александр а...,7770.txt |@word культурный код экономика эконо...
32230.txt,32230.txt,faq наука третий класс факт эксперимент резуль...,32230.txt |@word faq наука третий класс факт э...
27293.txt,27293.txt,обрушение волна поверхность жидкость математик...,27293.txt |@word обрушение волна поверхность ж...
481.txt,481.txt,существовать ли суперсимметрия мир элементарны...,481.txt |@word существовать ли суперсимметрия ...


### Coocs<a id="coocs"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Below we will make `vocab.txt` for computing word cooccurrences and compute coocs themselves.
Cooccurrences are needed for some coherence scores.

P.S.
The notebook [Making-Decorrelation-and-Topic-Selection-Friends.ipynb](https://github.com/machine-intelligence-laboratory/TopicNet/blob/master/topicnet/demos/Making-Decorrelation-and-Topic-Selection-Friends.ipynb) contains a bit more explanation and references concerning cooccurrences computation in ARTM library.

In [ ]:
DATASET._dictionary_file_path

'./Post_Science__internals__test/dict.dict'

We will need dictionary saved in human readable text format

In [ ]:
DICTIONARY_TXT_FILE_PATH = DATASET._dictionary_file_path + '.txt'

In [ ]:
DICTIONARY_TXT_FILE_PATH

'./Post_Science__internals/dict.dict.txt'

In [ ]:
dictionary = DATASET.get_dictionary()

In [ ]:
dictionary

artm.Dictionary(name=3ac8eb64-8702-4d53-8cab-ed55e4f9a715, num_entries=82162)

Let's do dictionary filtering now so as to remove too frequent words (and compute coocs only for meaningful words)

In [ ]:
dictionary.filter(min_df_rate=0.01, max_df_rate=0.5)

artm.Dictionary(name=3ac8eb64-8702-4d53-8cab-ed55e4f9a715, num_entries=5112)

Saving dictionary as text file and inside dataset entity

In [ ]:
dictionary.save_text(DICTIONARY_TXT_FILE_PATH)

DATASET._cached_dict = dictionary

Now we read dictionary entries to make a `vocab.txt`

In [ ]:
dictionary_lines = open(DICTIONARY_TXT_FILE_PATH, 'r').readlines()

In [ ]:
len(dictionary_lines)

5114

In [ ]:
dictionary_lines[:3]

['name: 3ac8eb64-8702-4d53-8cab-ed55e4f9a715 num_items: 3446\n',
 'token, class_id, token_value, token_tf, token_df\n',
 'социокультурный, @word, 3.8952195609454066e-05, 87.0, 39.0\n']

In [ ]:
dictionary_lines[-3:]

['легкий, @word, 8.909754978958517e-05, 199.0, 149.0\n',
 'строительный, @word, 2.999766729772091e-05, 67.0, 49.0\n',
 'внизу, @word, 1.8804508727043867e-05, 42.0, 36.0\n']

In [ ]:
vocab_text = ''

for line in dictionary_lines[2:]:
    token, modality, _, _, _ = line.strip().split(', ')
    vocab_text += f'{token} {modality}\n'

vocab_file_path = os.path.join(
    DATASET_INTERNALS_FOLDER_PATH,
    'vocab.txt',
)

with open(vocab_file_path, 'w') as f:
    f.write(vocab_text)


del vocab_text
del dictionary_lines

Now one must run BigARTM Command Line Utility to gather cooccerrences statistics.
To do so, there should be BigARTM executable file on the machine!
If `topicnet` was, for example, installed via `pip`, there may be no such ARTM executable.
However, one need exactly this one to run the command below.
There are some references abot BigARTM CLI here on [TopicNet's GitHub page](https://github.com/machine-intelligence-laboratory/TopicNet).

In [ ]:
# Specify path and run this cell, or run the command in bash

! ~/<YOUR PATH>/bigartm/build/bin/bigartm \
    -c $DATASET_INTERNALS_FOLDER_PATH/vw.txt \
    -v vocab.txt \
    --cooc-window 10 \
    --cooc-min-tf 2 \
    --write-cooc-tf cooc_tf_ \
    --cooc-min-df 2 \
    --write-cooc-df cooc_df_ \
    --write-ppmi-tf ppmi_tf_ \
    --write-ppmi-df ppmi_df_

The function below is supposed to transform `ppmi_tf_` or `ppmi_df_` contents in a format used in the notebook

In [ ]:
def transform_coocs_file(source_file_path, target_file_path, vocab_file_path):
    """
    Source file is assumed to be either `ppmi_tf_` or `ppmi_df_`

    """
    num_times_to_log = 10

    vocab = open(vocab_file_path, 'r').readlines()
    vocab = [l.strip().split()[0] for l in vocab]
    
    cooc_values = dict()
    word_word_value_triples = set()
    
    lines = open(source_file_path, 'r').readlines()
    
    for i, l in enumerate(lines):
        if i % (len(lines) // num_times_to_log) == 0:
            print(f'{i:6d} lines out of {len(lines)}')
        
        l = l.strip()
        words = l.split()

        if words[0].startswith('@'):  # modality
                                      # (not always included:
                                      #  @default_class seems not used in vocab.txt)
            words = words[1:]         # excluding modality
        
        anchor_word = words[0]
        other_word_values = words[1:]
        
        for word_and_value in other_word_values:
            other_word, value = word_and_value.split(':')
            value = float(value)
            
            cooc_values[(anchor_word, other_word)] = value
            
            # No need to do so: coherence scores do similar thing under the hood
            # cooc_values[(other_word, anchor_word)] = value  # if assume cooc values to be symmetric
            
            word_word_value_triples.add(
                tuple([
                    tuple(sorted([
                        vocab.index(anchor_word),
                        vocab.index(other_word)
                    ])),
                    value
                ])
            )
    
    new_text = ''
    
    for (w1, w2), v in word_word_value_triples:
        new_text += f'{w1} {w2} {v}\n'
    
    with open(target_file_path, 'w') as f:
        f.write(''.join(new_text))
    
    return cooc_values

In [ ]:
COOC_VALUES = transform_coocs_file(
    os.path.join(DATASET_INTERNALS_FOLDER_PATH, 'ppmi_tf_'),
    os.path.join(DATASET_INTERNALS_FOLDER_PATH, 'new_ppmi_tf_'),
    os.path.join(DATASET_INTERNALS_FOLDER_PATH, 'vocab.txt'),
)

     0 lines out of 1673
   167 lines out of 1673
   334 lines out of 1673
   501 lines out of 1673
   668 lines out of 1673
   835 lines out of 1673
  1002 lines out of 1673
  1169 lines out of 1673
  1336 lines out of 1673
  1503 lines out of 1673
  1670 lines out of 1673


In [ ]:
len(COOC_VALUES)

465882

In [ ]:
it = iter(COOC_VALUES.items())

print(next(it), next(it))

del it

(('както', 'понимать'), 0.697806) (('както', 'сталкиваться'), 0.388971)


If there are too many coocs (more than $1\,000\,000$ let's say) — better do some additional filtering.
For example, keep only coocs with numeric value bigger than some threshold

In [ ]:
min(COOC_VALUES.values())

1.67684e-07

In [ ]:
max(COOC_VALUES.values())

6.46194

In [ ]:
threshold_cooc = np.percentile(list(COOC_VALUES.values()), 10)

In [ ]:
threshold_cooc

0.08462922

In [ ]:
COOC_VALUES = {
    k: v for i, (k, v) in enumerate(COOC_VALUES.items()) if v >= threshold_cooc
}

del threshold_cooc

In [ ]:
len(COOC_VALUES)

419293

Saving coocs on disk (in case notebook crashes, for example) and than loading them to check if all OK

In [ ]:
COOC_VALUES_FILE_PATH = os.path.join(
    DATASET_INTERNALS_FOLDER_PATH, 'cooc_values.json'
)

with open(COOC_VALUES_FILE_PATH, 'w') as f:
    f.write(json.dumps(list(COOC_VALUES.items())))

In [ ]:
len(COOC_VALUES)

419293

In [ ]:
del COOC_VALUES

In [ ]:
if not os.path.isfile(COOC_VALUES_FILE_PATH):
    COOC_VALUES = dict()
else:
    raw_cooc_values = json.loads(open(COOC_VALUES_FILE_PATH, 'r').read())
    
    print(
        raw_cooc_values[:20]
    )

    COOC_VALUES = {
        tuple(d[0]): d[1] for d in raw_cooc_values
    }

[[['както', 'понимать'], 0.697806], [['както', 'сталкиваться'], 0.388971], [['както', 'переводить'], 0.543726], [['както', 'любовь'], 0.905235], [['както', 'возможно'], 0.342641], [['както', 'экономист'], 0.119018], [['както', 'дорога'], 0.537558], [['както', 'академический'], 0.357577], [['както', 'проявлять'], 0.827184], [['както', 'проблема'], 0.464159], [['както', 'подавлять'], 0.581892], [['както', 'трудно'], 0.735254], [['както', 'действительно'], 0.263179], [['както', 'оставаться'], 0.162784], [['както', 'образовываться'], 0.712785], [['както', 'молодой'], 0.623452], [['както', 'пора'], 0.332543], [['както', 'очевидно'], 0.415827], [['както', 'насколько'], 0.429967], [['както', 'неделя'], 1.35179]]


In [ ]:
len(COOC_VALUES)

419293

In [ ]:
SAMPLE_COOC_KEY = list(COOC_VALUES.keys())[419126]

In [ ]:
SAMPLE_COOC_KEY

('прекрасно', 'огромный')

#### Lower Memory Consumption (or a Bit of Shamanism. Part 1)<a id="optimizing-memory"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

It so happened that coocs are not stored in the best way possible in the coherence scores in the module.
Let's try to fix it right here in the notebook.

In [ ]:
COOC_VALUES.__getitem__(SAMPLE_COOC_KEY)

0.126559

Collecting words in a vocabulary

In [ ]:
VOCABULARY = set()

for word_a, word_b in tqdm(COOC_VALUES.keys(), file=sys.stdout):
    VOCABULARY.add(word_a)
    VOCABULARY.add(word_b)

100%|██████████| 419293/419293 [00:00<00:00, 2343319.19it/s]


In [ ]:
len(VOCABULARY)

1676

In [ ]:
WORD_TO_INDEX = {
    w: i for i, w in enumerate(VOCABULARY)
}

In [ ]:
len(WORD_TO_INDEX)

1676

Here is a trick: we imitate coocs values, but using word indices instead of words.
So, we need to store only vocabulary + index pairs instead of word pairs.
If there are many coocs (more than $1\,000\,000$), this may help to optimize the situation.

In [ ]:
class CoocValues:
    def __init__(self, word2index, coocs):
        self._word2index = word2index
        self._coocs = coocs
        
        self._index2word = {
            i: w
            for w, i in self._word2index.items()
        }
    
    def _map_key(self, word_pair_key):
        word_a, word_b = word_pair_key
        index_a = self._word2index[word_a]
        index_b = self._word2index[word_b]

        return (index_a, index_b)

    def __getitem__(self, word_pair_key):
        return self._coocs[self._map_key(word_pair_key)]
    
    def __len__(self):
        return len(self._coocs)
    
    def has_key(self, word_pair_key):
        word_a, word_b = word_pair_key

        return (
            word_a in self._word2index and
            word_b in self._word2index and
            self._map_key(word_pair_key) in self._coocs
        )
    
    def __contains__(self, k):
        return self.has_key(k)
    
    def __iter__(self):
        for (index_a, index_b) in self._coocs:
            yield (
                self._index2word[index_a],
                self._index2word[index_b]
            )

In [ ]:
COOC_VALUES = CoocValues(WORD_TO_INDEX, _COOC_VALUES)

Checking if all OK

In [ ]:
COOC_VALUES[SAMPLE_COOC_KEY]

0.126559

In [ ]:
len(COOC_VALUES)

419293

In [ ]:
for c in COOC_VALUES:
    print(c)

    break

('както', 'понимать')


In [ ]:
del SAMPLE_COOC_KEY

### Dpcuments for Coherence Scores<a id="docs-for-cohs"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Now we need tp choose documents to be used by coherence scores.
Choosing just all documents is not a good choice, because things will be too slow.

Computing documents lengths (to select only not so small and not so big documents)

In [ ]:
lengths = DATASET._data['vw_text'].apply(lambda text: len(text.split()))

In [ ]:
%%time

if not KEEP_DATASET_IN_MEMORY:
    lengths = lengths.compute()
    lengths = lengths.values

CPU times: user 2.63 s, sys: 0 ns, total: 2.63 s
Wall time: 2.5 s


In [ ]:
%%time

if not KEEP_DATASET_IN_MEMORY:
    DATASET.get_vw_document(DATASET._data_index[0])
else:
    DATASET.get_vw_document(DATASET._data.index[0])

In [ ]:
median_length = np.median(lengths)
p25_length = np.percentile(lengths, 25)
p75_length = np.percentile(lengths, 75)

print(f'{p25_length:.2f} (25%) < {median_length:.2f} (median) < {p75_length:.2f} (75%)')

182.00 (25%) < 462.50 (median) < 871.75 (75%)


These documents are of ordinary lengths:

In [ ]:
ORDINARY_DOCUMENTS = [
    d for i, d in enumerate(DOCUMENTS)
    if (
        lengths[i] <= p75_length
        and
        lengths[i] >= p25_length
    )
]

In [ ]:
print(
    f'Num ordinary documents: {len(ORDINARY_DOCUMENTS)}'
    f' ({100 * len(ORDINARY_DOCUMENTS) / len(DOCUMENTS):.2f}%'
    f' of all {len(DOCUMENTS)})'
)

Num ordinary documents: 1728 (50.15% of all 3446)


Let's use only `TEXT_LENGTH_FOR_COHERENCE` words for coherence scores

In [ ]:
TEXT_LENGTH_FOR_COHERENCE = 25000
NUM_TEST_DOCUMENTS = int(TEXT_LENGTH_FOR_COHERENCE / median_length)

In [ ]:
NUM_TEST_DOCUMENTS

54

In [ ]:
print(median_length * NUM_TEST_DOCUMENTS)

24975.0


In [ ]:
del lengths
del median_length
del p25_length
del p75_length

As we are going to choose `NUM_TEST_DOCUMENTS` randomly, seems we need to do this several times and average the results.
In the experiments, three seeds were used (so three times selecting documents and running models training).

In [ ]:
seed = 11221963 # 11221963 # 42 # 0
random = np.random.RandomState(seed)

TEST_DOCUMENTS = random.choice(
    ORDINARY_DOCUMENTS, size=NUM_TEST_DOCUMENTS, replace=False
)
TEST_DOCUMENTS = list(TEST_DOCUMENTS)

del random

In [ ]:
TEST_DOCUMENTS[:10]

['59620.txt',
 '16645.txt',
 '47547.txt',
 '28543.txt',
 '34582.txt',
 '13548.txt',
 '32976.txt',
 '21798.txt',
 '48114.txt',
 '36253.txt']

#### Lower Time Consumption in Case of Big Datasets (or a Bit of Shamanism. Part 2)<a id="optimizing-time"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Currently, if dataset is initialized with `keep_in_memory=False`, getting document text takes a lot of time (which is not acceptable if we are computing coherence scores).
So, we need to do another trick: we retrieve text for documents *once* (for documents which we selected previously) and save the texts on disk.
And then we modify the dataset entity in such a way so it reads these saved texts.

This is how slow things are before we try to do anything

In [ ]:
%%time

DATASET.get_vw_document(TEST_DOCUMENTS[0])

CPU times: user 2.14 s, sys: 0 ns, total: 2.14 s
Wall time: 2.01 s


,vw_text
id,
59620.txt,59620.txt |@word магический заговор постсоветс...


Saving original `get_vw_document()` function just in case

In [ ]:
DATASET._original_get_vw_document = DATASET.get_vw_document

In [ ]:
CACHED_VW_TEXTS_FOLDER_PATH = os.path.join(
    DATASET_INTERNALS_FOLDER_PATH,
    'cached_vw_texts',
)

In [ ]:
os.makedirs(CACHED_VW_TEXTS_FOLDER_PATH, exist_ok=True)

In [ ]:
def cached_vw_text_file_path(doc_id):
    return os.path.join(
        CACHED_VW_TEXTS_FOLDER_PATH, f'{doc_id}.csv'
    )

Here we save text on disk.
This may take time (especially if `keep_in_memory=False`)!
Up to $1$ hour or even more.

In [ ]:
%%time

# Needed to raise RAM from 8 Gb to 16 Gb (and still 4 processes)
for doc_id in tqdm(TEST_DOCUMENTS, total=len(TEST_DOCUMENTS), file=sys.stdout):
    vw_doc = DATASET._original_get_vw_document(doc_id)
    vw_doc.to_csv(cached_vw_text_file_path(doc_id))

In [ ]:
def quick_cached_get_vw_document(self, document_id):
    return pd.read_csv(
        cached_vw_text_file_path(document_id),
        index_col=0,
    )

And here we replace `get_vw_document()` with a new function

In [ ]:
DATASET.get_vw_document = quick_cached_get_vw_document.__get__(DATASET, Dataset)

In [ ]:
DATASET._original_get_vw_document

<bound method Dataset.get_vw_document of <topicnet.cooking_machine.dataset.Dataset object at 0x7f24184c5fd0>>

In [ ]:
DATASET.get_vw_document

<bound method quick_cached_get_vw_document of <topicnet.cooking_machine.dataset.Dataset object at 0x7f24184c5fd0>>

Now much better!

In [ ]:
%%time

DATASET.get_vw_document(TEST_DOCUMENTS[0])

CPU times: user 3.08 ms, sys: 3.36 ms, total: 6.44 ms
Wall time: 5.42 ms


,vw_text
id,
59620.txt,59620.txt |@word магический заговор постсоветс...


## Experiment<a id="experiment"></a>

Finally we are getting to the main part!)

### Scores (for Topics and Models)<a id="scores"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Here we define a lot of scores (which mainly differ in initial parameters).

In [ ]:
WINDOW = 20
NUM_TOP_WORDS = 20
MAX_NUM_OUT_WORDS = 5

VERBOSE = False

Two core coherence scores

In [ ]:
MAIN_TOPIC_SCORE = IntratextCoherenceScore(
    name='intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_none',
    data=DATASET,
    documents=TEST_DOCUMENTS,
    text_type=TextType.VW_TEXT,
    computation_method=ComputationMethod.SEGMENT_WEIGHT,
    word_topic_relatedness=WordTopicRelatednessType.PWT,
    specificity_estimation=SpecificityEstimationMethod.NONE,
    max_num_out_of_topic_words=MAX_NUM_OUT_WORDS,
    window=WINDOW,
    verbose=VERBOSE,
)

OTHER_TOPIC_SCORES = [
    SophisticatedTopTokensCoherenceScore(
        name='top_tokens_coherence_score__tt_vw__wtrt_pwt__sem_none',
        data=DATASET,
        documents=TEST_DOCUMENTS,
        text_type=TextType.VW_TEXT,
        word_topic_relatedness=WordTopicRelatednessType.PWT,
        specificity_estimation=SpecificityEstimationMethod.NONE,
        num_top_words=NUM_TOP_WORDS,
        window=WINDOW,
        verbose=VERBOSE,
    )
]

Other coherence score variations

In [ ]:
text_type_ids = {
    TextType.VW_TEXT: 'vw',
}
computation_method_ids = {
    ComputationMethod.SEGMENT_WEIGHT: 'seg_weight',
    ComputationMethod.SEGMENT_LENGTH: 'seg_length',
    ComputationMethod.SUM_OVER_WINDOW: 'sow',
}
word_topic_relatedness_type_ids = {
    WordTopicRelatednessType.PWT: 'pwt',
    WordTopicRelatednessType.PTW: 'ptw',
}
specificity_estimation_method_ids = {
    SpecificityEstimationMethod.NONE: 'none',
    SpecificityEstimationMethod.AVERAGE: 'av',
    SpecificityEstimationMethod.MAXIMUM: 'max',
}


param_combinations_intratext = list(
    itertools.product(
        text_type_ids,
        computation_method_ids,
        word_topic_relatedness_type_ids,
        specificity_estimation_method_ids,
    )
)
param_combinations_intratext.remove(
    (
        TextType.VW_TEXT,
        ComputationMethod.SEGMENT_WEIGHT,
        WordTopicRelatednessType.PWT,
        SpecificityEstimationMethod.NONE
    )
)

param_combinations_top_tokens = list(
    itertools.product(
        text_type_ids,
        word_topic_relatedness_type_ids,
        specificity_estimation_method_ids,
    )
)
param_combinations_top_tokens.remove(
    (
        TextType.VW_TEXT,
        WordTopicRelatednessType.PWT,
        SpecificityEstimationMethod.NONE
    )
)


for param_combination in param_combinations_intratext:
    (text_type,
     computation_method,
     word_topic_relatedness,
     specificity_estimation) = param_combination

    name = (
        f'intratext_coherence_score'
        f'__tt_{text_type_ids[text_type]}'
        f'__cm_{computation_method_ids[computation_method]}'
        f'__wtrt_{word_topic_relatedness_type_ids[word_topic_relatedness]}'
        f'__sem_{specificity_estimation_method_ids[specificity_estimation]}'
    )

    OTHER_TOPIC_SCORES.append(
        IntratextCoherenceScore(
            name=name,
            data=DATASET,
            documents=TEST_DOCUMENTS,
            text_type=text_type,
            computation_method=computation_method,
            word_topic_relatedness=word_topic_relatedness,
            specificity_estimation=specificity_estimation,
            max_num_out_of_topic_words=MAX_NUM_OUT_WORDS,
            window=WINDOW,
            verbose=VERBOSE,
        )
    )


for param_combination in param_combinations_top_tokens:
    (text_type,
     word_topic_relatedness,
     specificity_estimation) = param_combination

    name = (
        f'top_tokens_coherence_score'
        f'__tt_{text_type_ids[text_type]}'
        f'__wtrt_{word_topic_relatedness_type_ids[word_topic_relatedness]}'
        f'__sem_{specificity_estimation_method_ids[specificity_estimation]}'
    )

    OTHER_TOPIC_SCORES.append(
        SophisticatedTopTokensCoherenceScore(
            name=name,
            data=DATASET,
            documents=TEST_DOCUMENTS,
            text_type=text_type,
            word_topic_relatedness=word_topic_relatedness,
            specificity_estimation=specificity_estimation,
            # word_cooccurrences=<Formatted Cooc Values>,  # it turned out that
                                                           # this one is not necessary
                                                           # for time optimization
            num_top_words=NUM_TOP_WORDS,
            window=WINDOW,
            verbose=VERBOSE,
        )
    )

In [ ]:
len(OTHER_TOPIC_SCORES)

23

Another implementation of top-tokens-based coherence

In [ ]:
# Currently this score is particularly slow
# So we use just one combination of parameters
#
# param_combinations_other_top_tokens = list(
#     itertools.product([True, False], ['median', 'mean'], [None, 1e-7])
# )

param_combinations_other_top_tokens = list(
    itertools.product([True], ['median'], [1e-7])
)

if len(COOC_VALUES) > 0:  # with pre-computed coocs
    for param_combination in param_combinations_other_top_tokens:
        (kernel,
         average,
         active_topic_threshold) = param_combination

        name = (
            f'top_tokens_coherence_other_implementation_score'
            f'__ker_{kernel}'
            f'__av_{average}'
            f'__att_{active_topic_threshold}'
        )

        OTHER_TOPIC_SCORES.append(
            SimpleTopTokensCoherenceScore(
                name=name,
                data=DATASET,
                cooccurrence_values=COOC_VALUES,
                num_top_tokens=20,
                kernel=kernel,
                average=average,
                active_topic_threshold=active_topic_threshold,
            )
        )

In [ ]:
len(OTHER_TOPIC_SCORES)

24

And a pair of default ARTM scores (these ones are fast)

In [ ]:
OTHER_SCORES = [
    SparsityPhiScore(
        name='sparsity_phi_score'
    ),
    SparsityThetaScore(
        name='sparsity_theta_score'
    ),
]

### Bank Creation<a id="bank-creation"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Here we finally run the experiment!

In [ ]:
# We use only one train function here
# Other variations are also possible
# It would be even better to make bank using several train functions
# However, it would also take way more time 

TRAIN_FUNCS = default_train_func  # default train func

In [ ]:
DATASET_INTERNALS_FOLDER_PATH

'./Post_Science__internals__test'

In [ ]:
SEARCH_RESULTS_FOLDER_PATH = os.path.join(
    DATASET_INTERNALS_FOLDER_PATH, 'result'
)

# File with some info about the process
SEARCH_RESULT_FILE_PATH = os.path.join(
    SEARCH_RESULTS_FOLDER_PATH, f'search_result__{seed}.json'
)

# Bank, with topics and their score values
BANK_FOLDER_PATH = os.path.join(
    SEARCH_RESULTS_FOLDER_PATH, f'bank__{seed}'
)

In [ ]:
SEARCH_RESULTS_FOLDER_PATH

'./Post_Science__internals__test/result'

In [ ]:
BANK_FOLDER_PATH

'./Post_Science__internals__test/result/bank__11221963'

In [ ]:
os.makedirs(SEARCH_RESULTS_FOLDER_PATH, exist_ok=True)
os.makedirs(BANK_FOLDER_PATH, exist_ok=True)

In [ ]:
seed

11221963

One cay vary some parameters below (for example `max_num_models` and `num_fit_iterations`).

In [ ]:
optimizer = TopicBankMethod(
    data        = DATASET,
    min_df_rate = 0.0,  # dictionary filtering has already been done little earlier
    max_df_rate = 1.0,  #   so we don't want these parameters to have any effect

    main_topic_score   = MAIN_TOPIC_SCORE,
    other_topic_scores = OTHER_TOPIC_SCORES,
    other_scores       = OTHER_SCORES,
    documents          = TEST_DOCUMENTS,

    start_model_number   = 0,
    max_num_models       = 20,
    one_model_num_topics = 100,
    num_fit_iterations   = 100,  # 100 should be enough;
                                 # however, for big data better to reduce this one
                                 # (otherwise the process will be too slow)

    topic_score_threshold_percentile = 90,

    save_bank         = True,
    save_model_topics = True,
    save_file_path    = SEARCH_RESULT_FILE_PATH,
    bank_folder_path  = BANK_FOLDER_PATH,

    train_funcs = TRAIN_FUNCS,
    
    verbose = True,
)

# TODO: use Holdout Perplexity as Stop score

In [ ]:
optimizer._result.keys()

dict_keys(['optimum', 'optimum_std', 'bank_scores', 'bank_topic_scores', 'model_scores', 'model_topic_scores', 'num_bank_topics', 'num_model_topics'])

Checking file paths

In [ ]:
optimizer._save_file_path

'./Post_Science__internals__test/result/search_result__11221963.json'

In [ ]:
optimizer._topic_bank._path

'./Post_Science__internals__test/result/bank__11221963'

Fulfilling the search (get ready for a really long process!):

In [ ]:
%%time

optimizer.search_for_optimum(dataset)

 96%|█████████▌| 24/25 [44:07<02:26, 146.57s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



  5%|▌         | 1/20 [54:54<17:23:11, 3294.29s/it]
...................................................
...................more models....................
...................................................
100%|██████████| 20/20 [21:39:25<00:00, 3898.28s/it]
CPU times: user 1d 10h 46s, sys: 3h 46min 9s, total: 1d 13h 46min 55s
Wall time: 21h 39min 25s


What topics we have in bank

In [ ]:
optimizer._topic_bank.view_topics().head()

topic_0  topic_1  topic_2  topic_3  topic_4  topic_5  \
@word перикл          0.0      0.0      0.0      0.0      0.0      0.0   
      леагр           0.0      0.0      0.0      0.0      0.0      0.0   
      ольвийский      0.0      0.0      0.0      0.0      0.0      0.0   
      амфора          0.0      0.0      0.0      0.0      0.0      0.0   
      вазопись        0.0      0.0      0.0      0.0      0.0      0.0   

                  topic_6  topic_7  topic_8  topic_9  
@word перикл          0.0      0.0      0.0      0.0  
      леагр           0.0      0.0      0.0      0.0  
      ольвийский      0.0      0.0      0.0      0.0  
      амфора          0.0      0.0      0.0      0.0  
      вазопись        0.0      0.0      0.0      0.0

In [ ]:
bank_topics = optimizer._topic_bank.view_topics()

In [ ]:
bank_topics.shape

(2514, 15)

In [ ]:
bank_topics.head()

topic_0  topic_1       topic_2  \
@default_class както                  0.0  0.00000  5.497794e-11   
               гравитационный         0.0  0.00000  0.000000e+00   
               жена                   0.0  0.00139  0.000000e+00   
               продолжительность      0.0  0.00000  0.000000e+00   
               одновременно           0.0  0.00075  5.159791e-06   

                                       topic_3       topic_4       topic_5  \
@default_class както              0.000000e+00  1.985204e-08  2.223492e-14   
               гравитационный     0.000000e+00  0.000000e+00  0.000000e+00   
               жена               0.000000e+00  0.000000e+00  1.217113e-04   
               продолжительность  3.470690e-15  0.000000e+00  0.000000e+00   
               одновременно       7.805681e-09  3.031660e-06  5.124066e-09   

                                   topic_6       topic_7   topic_8   topic_9  \
@default_class както              0.000009  0.000000e+00  0.000000  0.000000   
               гравитационный     0.000000  0.000000e+00  0.000000  0.022702   
               жена               0.000000  3.405896e-15  0.000000  0.000000   
               продолжительность  0.000000  4.282006e-04  0.000000  0.000000   
               одновременно       0.000146  8.667931e-13  0.000935  0.000564   

                                      topic_10  topic_11  topic_12  \
@default_class както              0.000000e+00       0.0       0.0   
               гравитационный     0.000000e+00       0.0       0.0   
               жена               0.000000e+00       0.0       0.0   
               продолжительность  3.474610e-08       0.0       0.0   
               одновременно       1.033011e-07       0.0       0.0   

                                      topic_13      topic_14  
@default_class както              2.414336e-06  4.583967e-11  
               гравитационный     0.000000e+00  0.000000e+00  
               жена               1.091175e-13  8.552548e-05  
               продолжительность  0.000000e+00  0.000000e+00  
               одновременно       2.076379e-05  1.407077e-09

In [ ]:
bank_topics['topic_14'].sort_values(ascending=False)[:20]

@default_class  век             0.229964
                xx              0.053898
                xix             0.051478
                начало          0.043227
                первый          0.038318
                конец           0.035413
                второй          0.019743
                середина        0.018641
                время           0.017661
                половина        0.017191
                xviii           0.013792
                классический    0.013726
                xvii            0.012661
                возникать       0.011614
                эпоха           0.010723
                хх              0.010398
                новый           0.010231
                столетие        0.008743
                образ           0.008709
                период          0.008326
Name: topic_14, dtype: float64

And topic scores

In [ ]:
optimizer._topic_bank.view_topic_scores()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
kernel_size,229.000000,408.000000,102.000000,157.000000,235.000000,302.000000,345.000000,347.000000,283.000000,391.000000,338.000000,59.000000,121.000000,407.000000,214.000000
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_none,0.129687,0.123335,0.468014,0.249535,0.211076,0.246103,0.084003,0.103003,0.149715,0.085349,0.102518,0.177264,0.116741,0.102491,0.099711
top_tokens_coherence_score__tt_vw__wtrt_pwt__sem_none,1.133654,0.667880,1.629793,0.835258,0.835292,0.806799,0.434416,0.921406,0.700194,0.409935,0.473712,0.614014,0.583990,0.500487,0.975982
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_av,0.127684,0.121866,0.462997,0.246569,0.207621,0.243621,0.082943,0.101184,0.147921,0.084183,0.101309,0.173868,0.113667,0.101379,0.097300
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_max,0.120821,0.114016,0.462851,0.245817,0.116343,0.245570,0.079723,0.083172,0.143387,0.075810,0.096531,0.103924,0.100431,0.100097,0.080213
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_none,0.463680,0.783485,0.763705,0.481053,0.594415,0.914586,0.645213,0.466021,0.734765,0.765880,0.903625,0.845633,0.492089,0.642143,0.650835
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_av,0.452750,0.771956,0.753009,0.470381,0.583590,0.903922,0.633082,0.455416,0.723801,0.753463,0.890064,0.830870,0.479616,0.631021,0.636409
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_max,0.385764,0.677018,0.718970,0.418170,0.345956,0.879365,0.549258,0.345888,0.661878,0.648797,0.772138,0.636360,0.417972,0.574683,0.525002
intratext_coherence_score__tt_vw__cm_seg_length__wtrt_pwt__sem_none,1.092971,1.152834,1.069599,1.067143,1.082481,1.066313,1.213043,1.060516,1.096398,1.241758,1.356061,1.476266,1.247272,1.112245,1.442545
intratext_coherence_score__tt_vw__cm_seg_length__wtrt_pwt__sem_av,1.092971,1.152834,1.069599,1.067143,1.082481,1.066313,1.213043,1.060516,1.096398,1.241758,1.356061,1.476266,1.247272,1.112245,1.442545


All models are also saved (topics as $\Phi$ matrices and topic score values)

In [ ]:
! ls $optimizer._topic_bank._path

model_0__phi.bin	   model_4__phi.bin	      model_8__phi.bin
model_0__topic_scores.bin  model_4__topic_scores.bin  model_8__topic_scores.bin
model_1__phi.bin	   model_5__phi.bin	      model_9__phi.bin
model_1__topic_scores.bin  model_5__topic_scores.bin  model_9__topic_scores.bin
model_2__phi.bin	   model_6__phi.bin	      topics.bin
model_2__topic_scores.bin  model_6__topic_scores.bin  topic_scores.bin
model_3__phi.bin	   model_7__phi.bin
model_3__topic_scores.bin  model_7__topic_scores.bin


## Postprocessing<a id="postprocessing"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

Remember, we were going to do computations several times (with different seeds and documents used by coherence scores).
Here we combine these results to get just one topic bank.
Supposedly, topic banks for different seeds should *not* differ much: exactly the same models are used in all cases.
What is different is just the way we estimate topics quality (or, better, the documents we use for topic coherence computation).

P.S.
As I am writing the explanatory .md text, I realize that bank creation process could have been optimized: models should have been trained *only once* for each dataset.
And then, for different seeds, would go quality estimation without actual training.
In the version presented in the notebook there are model training for each seed.
However, coherence computation still remains the most time consuming part (and this one we should be done separately for each seed).

In [ ]:
os.listdir(DATASET_INTERNALS_FOLDER_PATH)

['vw.txt',
 'vocab.txt',
 'dict.dict.txt',
 'new_ppmi_tf_',
 'ppmi_tf_',
 'dict.dict',
 'cooc_values.json',
 'batches',
 'result']

In [ ]:
SEARCH_RESULTS_FOLDER_PATH

'./Post_Science__internals/result'

In [ ]:
os.listdir(SEARCH_RESULTS_FOLDER_PATH)

['bank__11221963',
 'search_result__11221963.json',
 'bank__0',
 'search_result__0.json',
 'search_result__42.json',
 'bank__42']

Looking what is insides one bank's folder

In [ ]:
one_dataset_bank_folder_path = os.path.join(
    SEARCH_RESULTS_FOLDER_PATH,
    'bank__11221963',
)

folder_contents = sorted(os.listdir(one_dataset_bank_folder_path))

print(folder_contents[:10])
print('...')
print(folder_contents[-10:])

['model_0__phi.bin', 'model_0__theta.bin', 'model_0__topic_scores.bin', 'model_10__phi.bin', 'model_10__theta.bin', 'model_10__topic_scores.bin', 'model_11__phi.bin', 'model_11__theta.bin', 'model_11__topic_scores.bin', 'model_12__phi.bin']
...
['model_7__theta.bin', 'model_7__topic_scores.bin', 'model_8__phi.bin', 'model_8__theta.bin', 'model_8__topic_scores.bin', 'model_9__phi.bin', 'model_9__theta.bin', 'model_9__topic_scores.bin', 'topic_scores.bin', 'topics.bin']


What info is in one result file

In [ ]:
one_dataset_search_result_path = os.path.join(
    SEARCH_RESULTS_FOLDER_PATH,
    'search_result__11221963.json',
)

one_search_result = json.loads(open(one_dataset_search_result_path, 'r').read())

print(one_search_result.keys())

dict_keys(['optimum', 'optimum_std', 'bank_scores', 'bank_topic_scores', 'model_scores', 'model_topic_scores', 'num_bank_topics', 'num_model_topics'])


What result fields we are interested in (for combining)

In [ ]:
class SearchResultKey(Enum):
    BANK_SCORES = 'bank_scores'
    BANK_TOPIC_SCORES = 'bank_topic_scores'
    MODEL_SCORES = 'model_scores'
    MODEL_TOPIC_SCORES = 'model_topic_scores'
    NUM_BANK_TOPICS = 'num_bank_topics'
    NUM_MODEL_TOPICS = 'num_model_topics'

How many items are there in each result field

In [ ]:
for search_result_key in SearchResultKey:
    print(search_result_key.value)
    print(' ' * 4 + str(len(one_search_result[search_result_key.value])) + ' items')

bank_scores
    20 items
bank_topic_scores
    15 items
model_scores
    20 items
model_topic_scores
    20 items
num_bank_topics
    20 items
num_model_topics
    20 items


Example of items in result fields

In [ ]:
for i, search_result_key in enumerate([
        SearchResultKey.BANK_SCORES,
        SearchResultKey.BANK_TOPIC_SCORES,
        SearchResultKey.MODEL_SCORES]):

    if i > 0:
        print()

    print(search_result_key.value)
    print(' ' * 4 + str(len(one_search_result[search_result_key.value])) + ' items')
    print(' ' * 4 + 'sample item: ', end='')
    
    one_item = next(iter(one_search_result[search_result_key.value]))
    
    print(
        '{'
        + ', '.join(
            [f'{k}: {v}' for k, v in list(one_item.items())[:3]]
        )
        + ', ...'
        + '}'
    )

bank_scores
    20 items
    sample item: {perplexity_score: 1150.075439453125, sparsity_phi_score: 0.5031424164772034, sparsity_theta_score: 0.005107370670884848, ...}

bank_topic_scores
    15 items
    sample item: {kernel_size: 229, intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_none: 0.1296872250691411, top_tokens_coherence_score__tt_vw__wtrt_pwt__sem_none: 1.133653577000209, ...}

model_scores
    20 items
    sample item: {perplexity_score: 739.5682373046875, sparsity_phi_score: 0.48647573590278625, sparsity_theta_score: 0.009358677081763744, ...}


In [ ]:
search_result_key = SearchResultKey.MODEL_TOPIC_SCORES

print(search_result_key.value)
print(' ' * 4 + str(len(one_search_result[search_result_key.value])) + ' items')

one_item = next(iter(one_search_result[search_result_key.value]))

print(' ' * 4 + f'Item length: {len(one_item)}')
print(' ' * 4 + 'Sample item: ', end='')

print(
    '['
    + '{'
    + ', '.join(
        [f'{k}: {v}' for k, v in list(next(iter(one_item)).items())[:3]]
    )
    + ', ...'
    + '}'
    + ', ...'
    + ']'
)

model_topic_scores
    20 items
    Item length: 100
    Sample item: [{kernel_size: 397, intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_none: 0.017755509703420103, top_tokens_coherence_score__tt_vw__wtrt_pwt__sem_none: 0.3707505407301823, ...}, ...]


In [ ]:
for i, search_result_key in enumerate([
        SearchResultKey.NUM_BANK_TOPICS,
        SearchResultKey.NUM_MODEL_TOPICS]):

    if i > 0:
        print()

    print(search_result_key.value)
    
    result_value = one_search_result[search_result_key.value]
    
    print(' ' * 4 + f'Length: {len(result_value)}')
    print(
        ' ' * 4
        + '['
        + ', '.join(str(n) for n in result_value[:5])
        + ', ..., '
        + ', '.join(str(n) for n in result_value[-5:])
        + ']'
    )

num_bank_topics
    Length: 20
    [10, 10, 11, 11, 11, ..., 15, 15, 15, 15, 15]

num_model_topics
    Length: 20
    [100, 100, 100, 100, 100, ..., 100, 100, 100, 100, 100]


Loading banks corresponding to seeds (if there is a bank for such seed)

In [ ]:
POSSIBLE_BANK_SEEDS = [11221963, 42, 0]

In [ ]:
SEED_TO_BANK = dict()

for seed in POSSIBLE_BANK_SEEDS:
    one_dataset_bank_folder_path = os.path.join(
        SEARCH_RESULTS_FOLDER_PATH,
        f'bank__{seed}',
    )
    
    if not os.path.isdir(one_dataset_bank_folder_path):
        print(f'No bank for such seed: {seed}.'
              f' Folder "{one_dataset_bank_folder_path}" doesn\'t exist')
        
        continue
    
    SEED_TO_BANK[seed] = TopicBank(
        save=False,
        save_folder_path=one_dataset_bank_folder_path,
    )


BANK_SEEDS = list(SEED_TO_BANK.keys())

Seeds for which there is a bank saved on disk

In [ ]:
BANK_SEEDS

[11221963, 42, 0]

Number of topics in bank

In [ ]:
len(
    next(iter(SEED_TO_BANK.values())).topics
)

15

This is also number of topics, but some items in this list are `None` (when a topic is excluded from bank)

In [ ]:
len(
    next(iter(SEED_TO_BANK.values()))._topics
)

17

Topic matrix

In [ ]:
next(iter(SEED_TO_BANK.values())).view_topics().shape

(2514, 15)

In [ ]:
SEED_TO_BANK[BANK_SEEDS[0]].view_topics()['topic_0'].sort_values(ascending=False)[:10]

@default_class  проблема        0.377909
                вопрос          0.051589
                решать          0.039914
                трудность       0.028569
                возникать       0.026642
                сталкиваться    0.022061
                другой          0.015737
                сложный         0.013366
                создавать       0.011581
                существовать    0.010672
Name: topic_0, dtype: float64

In [ ]:
SEED_TO_BANK[BANK_SEEDS[1]].view_topics()['topic_0'].sort_values(ascending=False)[:10]

@default_class  слово          0.193740
                словарь        0.035589
                буква          0.027122
                значение       0.022216
                речь           0.020765
                глагол         0.016013
                русский        0.012727
                конструкция    0.010727
                часто          0.009467
                пример         0.008037
Name: topic_0, dtype: float64

Let's compare banks corresponding to different seeds to see if they differ much or not (supposedly not)

In [ ]:
def compare_topics(
        bank_seed1: int,
        bank_seed2: int,
        max_num_twins_to_display: int = 20) -> None:

    topics_with_twin = set()
    
    print(
        f'Close topics:'
        f' <topic from "{bank_seed1}" bank>'
        f' <topic from "{bank_seed1}" bank>'
        f' <distance>'
    )
    
    num_displayed = 0
    
    for i1, t1 in enumerate(SEED_TO_BANK[bank_seed1].topics):
        for i2, t2 in enumerate(SEED_TO_BANK[bank_seed2].topics):
            d = TopicBankMethod._jaccard_distance(t1, t2)

            if d < 0.5:
                topics_with_twin.add(i1)
                num_displayed = num_displayed + 1
                
                print(f'{i1}\t{i2}\t{d}')
            
            if num_displayed >= max_num_twins_to_display:
                break

        if num_displayed >= max_num_twins_to_display:
            break

    print()

    max_seed_length = max([len(str(s)) for s in [bank_seed1, bank_seed2]])

    for seed in [bank_seed1, bank_seed2]:
        print(
            f'Num topics in bank "{seed:{max_seed_length}}":'
            f' {len(SEED_TO_BANK[seed].topics)}'
        )

    print(
        f'\nNum topics in bank "{bank_seed1}" with twins in bank "{bank_seed2}":'
        f' {len(topics_with_twin)}'
    )

In [ ]:
if len(BANK_SEEDS) >= 2:
    compare_topics(BANK_SEEDS[0], BANK_SEEDS[1])

Close topics: <topic from "11221963" bank> <topic from "11221963" bank> <distance>
1	1	7.030248077910528e-06
2	2	5.364596949331002e-06
3	4	6.971623467633137e-06
4	5	9.489878975976751e-06
5	6	3.474788023982711e-06
7	7	5.1163178486079985e-06
8	9	6.845627906426621e-06
11	14	1.107408409306565e-06
12	15	1.2886343224716157e-06
13	10	0.46749039824435024

Num topics in bank "11221963": 15
Num topics in bank "      42": 16

Num topics in bank "11221963" with twins in bank "42": 10


In [ ]:
if len(BANK_SEEDS) >= 3:
    compare_topics(BANK_SEEDS[1], BANK_SEEDS[2])

Close topics: <topic from "42" bank> <topic from "42" bank> <distance>
0	0	2.1602862702918557e-05
1	1	1.4857055578576528e-05
2	2	9.737322482217259e-06
4	3	1.0445701357442161e-05
5	4	1.668583528036116e-05
6	5	6.556589902784182e-06
7	7	9.581083945442437e-06
9	8	1.2234920047093922e-05
13	11	1.5169697592520848e-06
14	13	6.072632612319495e-07
15	14	1.1275848412761746e-06

Num topics in bank "42": 16
Num topics in bank " 0": 15

Num topics in bank "42" with twins in bank "0": 11


And here we combine banks to get just one bank!

In [ ]:
COMBINED_BANK_TOPIC_INDICES = dict()

assert len(BANK_SEEDS) >= 1

COMBINED_BANK_TOPIC_INDICES[BANK_SEEDS[0]] = list(
    range(len(SEED_TO_BANK[BANK_SEEDS[0]].topics))
)


def get_final_bank_topics() -> Iterable[dict]:  # a token in dict may be as just one word,
                                                # or a tuple of words
    for seed, indices in COMBINED_BANK_TOPIC_INDICES.items():
        for i in indices:
            yield SEED_TO_BANK[seed].topics[i]


def get_final_bank_topic_scores() -> Iterable[Dict[str, float]]:
    for seed, indices in COMBINED_BANK_TOPIC_INDICES.items():
        for i in indices:
            yield SEED_TO_BANK[seed].topic_scores[i]


DISTANCE_THRESHOLD = 0.5  # if topics in banks are really similar, this will be enough

if len(BANK_SEEDS) == 1:
    pass
else:
    for seed in tqdm(BANK_SEEDS[1:], total=len(BANK_SEEDS) - 1, file=sys.stdout):
        COMBINED_BANK_TOPIC_INDICES[seed] = list()

        for i, t in enumerate(SEED_TO_BANK[seed].topics):
            if any([TopicBankMethod._jaccard_distance(t, bank_topic) < DISTANCE_THRESHOLD
                    for bank_topic in get_final_bank_topics()]):

                continue

            COMBINED_BANK_TOPIC_INDICES[seed].append(i)

In [ ]:
print('{0:>10}\t{1:>10}'.format('seed', 'num_topics'))
print('-' * 30)

for seed, topic_indices in COMBINED_BANK_TOPIC_INDICES.items():
    print(f'{seed:10}\t{len(topic_indices):10}')

print('-' * 30)

print(
    ' ' * 10
    + '\t'
    + f'{sum(len(inds) for inds in COMBINED_BANK_TOPIC_INDICES.values()):10}'
)

      seed	num_topics
------------------------------
  11221963	        15
        42	         6
         0	         3
------------------------------
          	        24


P.S. Output of the previous cell for all datasets:

```
PostNauka:

seed	num_topics
------------------------------
  11221963	        15
        42	         6
         0	         3
------------------------------
          	        24

Reuters:

seed	num_topics
------------------------------
  11221963	        22
        42	        10
         0	         1
------------------------------
          	        33

Brown:

seed	num_topics
------------------------------
  11221963	        23
        42	        16
         0	         8
------------------------------
          	        47

20 NG:

seed	num_topics
------------------------------
  11221963	        27
        42	        13
         0	        15
------------------------------
          	        55

AG News:

seed	num_topics
------------------------------
  11221963	        37
        42	         7
         0	         6
------------------------------
          	        50

Watan:

     seed	num_topics
------------------------------
  11221963	        10
        42	         5
         0	         1
------------------------------
          	        16

Habrahabr:

seed	num_topics
------------------------------
  11221963	        11
        42	        11
------------------------------
          	        22
```

Saving the bank

In [ ]:
COMBINED_SEARCH_RESULT_FOLDER_PATH = os.path.join(
    DATASET_INTERNALS_FOLDER_PATH,
    'result_combined',
)

COMBINED_BANK_FOLDER_PATH = os.path.join(
    COMBINED_SEARCH_RESULT_FOLDER_PATH,
    'bank',
)

In [ ]:
COMBINED_SEARCH_RESULT_FOLDER_PATH

'./Post_Science__internals__test/result_combined'

In [ ]:
COMBINED_BANK_FOLDER_PATH

'./Post_Science__internals__test/result_combined/bank'

In [ ]:
os.makedirs(COMBINED_SEARCH_RESULT_FOLDER_PATH, exist_ok=True)
os.makedirs(COMBINED_BANK_FOLDER_PATH, exist_ok=True)

In [ ]:
COMBINED_BANK = TopicBank(
    save=False,
    save_folder_path=COMBINED_BANK_FOLDER_PATH,
)

In [ ]:
COMBINED_BANK._topics = list(get_final_bank_topics())
COMBINED_BANK._topic_scores = list(get_final_bank_topic_scores())

Checking if all OK

In [ ]:
assert len(COMBINED_BANK._topics) == len(COMBINED_BANK._topic_scores)
assert len(COMBINED_BANK._topics) == sum(
    len(v) for v in COMBINED_BANK_TOPIC_INDICES.values()
)

In [ ]:
os.listdir(COMBINED_BANK_FOLDER_PATH)

[]

In [ ]:
COMBINED_BANK.save()

In [ ]:
os.listdir(COMBINED_BANK_FOLDER_PATH)

['topic_scores.bin', 'topics.bin']

In [ ]:
del COMBINED_BANK
del COMBINED_BANK_TOPIC_INDICES

Now topic banks are ready (all info is in `COMBINED_BANK_FOLDER_PATH` for each dataset).
Notebook [TopicBank-Experiment: Model Validation](TopicBank-Experiment-ModelValidation.ipynb) goes next: we are going to estimate topic models quality with the help of topic bank.